In [1]:
%pylab inline
import numpy as np
import pickle as cp

Populating the interactive namespace from numpy and matplotlib


In [2]:
data = cp.load(open('gug.pkl','rb'))


FileNotFoundError: [Errno 2] No such file or directory: 'gug.pkl'

In [ ]:
data = data.transpose(2,1,0)

In [ ]:
import pylab as plt

In [ ]:
plt.plot(data[0][0],data[0][1])
plt.plot(data[1][0],data[1][1])
plt.plot(data[2][0],data[2][1])
plt.plot(data[4][0],data[4][1])

## Histograms of increments

In [ ]:
increments = np.diff(data,axis=-1)

In [ ]:
# increments lenght
dl = (increments**2).sum(axis=1)**.5

In [ ]:
dl.shape

In [ ]:
def gumbell(x,beta,mu):
    z = (x-mu)/beta
    g = 1./beta * exp(-(z+exp(-z)))
    return g

from scipy.optimize import curve_fit

In [ ]:
count,edges= np.histogram(dl.flatten(),bins=50,normed=1)
edges = (edges[:-1]+edges[1:])/2.
plt.semilogy(edges,count)
# plt.plot(edges,exp(-edges*8)*1e1)
(beta,mu),_ = curve_fit(gumbell,edges,count)
plt.plot(edges,gumbell(edges,beta,mu),label='gumbell law')

print("--")
print("mu",mu)
print("beta",beta)
plt.grid()
plt.xlabel('increment lenght')
plt.ylabel('pdf')
plt.legend()

I see a exponential decay above.. which is not good for a long tails 
as in LWs

In [ ]:
increments.shape

In [ ]:
def gaus(x,a,x0,sigma):
    return a*exp(-(x-x0)**2/(2*sigma**2))

def meanstd_no_outliers(x,sigmas=3):
    mean = x.mean()
    std = x.std()
    idx = (x>(mean-sigmas*std) ) & (x<(mean+sigmas*std))

    mean = x[idx].mean()
    std = x[idx].std()
    return mean,std



In [ ]:
for j in [0,1]: # 0 stands for increments in x direction, 1 for y 
    X = increments[:,j,:].flatten() 
    count,edges= np.histogram(X,bins=np.arange(-1.5,1.5,.05))
    edges = (edges[:-1]+edges[1:])/2.
    plt.semilogy(edges,count)
    
#     mean = X.mean()
#     sigma = X.std()
    
    mean,sigma = meanstd_no_outliers(X)
    
    plt.plot(edges,gaus(edges,count.max(),mean,sigma),'--')
    
    plt.ylim(.1,6e4)
    plt.grid(1)

In [ ]:
# area below the gaussian = sqrt(2)*sqrt(%pi)*a*sigma
gauss_area = (2*np.pi)**.5*count.max()*sigma
total_counts = sum(count)*.05
non_gauss = total_counts - gauss_area
print("gauss_area",gauss_area)
print("total_counts",total_counts)
print("non_gauss",non_gauss)

Finally, heavy tails!
But the estimate needs some work bcs we have one hyper param

## brutal approach based on velocity

In [ ]:
velocity = np.diff(data,axis=-1)
speed = (velocity**2).sum(axis=1)**.5
speed.shape

In [ ]:
from scipy.ndimage.filters import gaussian_filter1d

In [ ]:
plt.plot(gaussian_filter1d(speed[0],sigma=1))
plt.plot(gaussian_filter1d(speed[0],sigma=50))

In [ ]:
# direction vector
d = velocity/speed[:,np.newaxis,:]
p = (d[:,:,:-1]*d[:,:,1:]).sum(axis=1)

In [ ]:
plt.plot(gaussian_filter1d(p[0],sigma=1))
plt.plot(gaussian_filter1d(p[0],sigma=50))

In [ ]:
plt.scatter(p.flatten(),speed[:,1:].flatten(),s=1,alpha=.1);
plt.xlabel('orientation wrt prev step')
plt.ylabel('velocity')

In [ ]:
plt.hist2d(p.flatten(),speed[:,1:].flatten(),bins=25);
plt.xlabel('orientation wrt prev step')
plt.ylabel('velocity')

So, faster correlates to  more aligned consecutive steps, which makes sense.
And the visualization confirms

In [ ]:
for j in range(5):
    plt.scatter(data[j][0],data[j][1],s=.5,c=plt.cm.jet(plt.Normalize()(gaussian_filter1d(speed[j],sigma=50))))
plt.title('color coded by velocity')

In [ ]:
for j in range(5):
    plt.scatter(data[j][0],data[j][1],s=.5,c=plt.cm.jet(plt.Normalize()(gaussian_filter1d(p[j],sigma=50))))


One question is: what's the prob that two steps are aligned at given velocity $P(\theta|v)$, this can be computed from above, and used to split the trajectories

In [ ]:
from scipy.ndimage.filters import median_filter

In [ ]:
#hiperparameters
sigma = 50
threshold = .5 # or 50% of signal

plt.figure(figsize=(10,10))
for j in range(3):

    plt.scatter(data[j][0],data[j][1],s=.5,c=plt.cm.jet(plt.Normalize()(gaussian_filter1d(p[j],sigma=50))))
    
    mask = (plt.Normalize()(gaussian_filter1d(p[j],sigma=sigma))>threshold).astype(float)
    plt.scatter(data[j][0],data[j][1],s=.1,c=plt.cm.gray_r(mask))
    

In [ ]:
#hiperparameters
sigma = 50
threshold = .5 # or 50% of signal
def lag_times_hist(sigma,threshold):
    running = []
    diffusing = []
    for j in range(50):
        mask = (plt.Normalize()(gaussian_filter1d(p[j],sigma=sigma))>threshold).astype(float)
        # and now we compute the lag times .. guess how?!
        true_to_false = np.diff(mask)
        where_is_it = np.where(true_to_false)
        lags = np.diff(where_is_it)[0]
        other = lambda x: {1:0,0:1}[x]
        assigned_lags = {mask[0]:lags[::2],other(mask[0]):lags[1::2]}
        running.extend(assigned_lags[1])
        diffusing.extend(assigned_lags[0])
    count,edges = np.histogram(running,bins=np.arange(0,5000,100));
    edges = (edges[1:]+edges[:-1])/2
    plt.semilogy(edges,count,label=str(sigma)+' '+str(threshold))
    plt.xlabel('lag times of running motion')
    plt.legend(loc='best')
    plt.grid(1)

threshold = .7
    
lag_times_hist(100,threshold)
lag_times_hist(50,threshold)
lag_times_hist(30,threshold)
lag_times_hist(10,threshold)
lag_times_hist(5,threshold)
lag_times_hist(1,threshold)

Dunno if really feasible to split the path in diffusive and non diffusive in this way

## autocorrelation of speed

In [ ]:
from scipy.signal import correlate

In [ ]:
velocity.shape,speed.shape

In [ ]:
speed_detrended = speed-speed.mean(axis=1)[:,np.newaxis]

In [ ]:
def acorr(x):
    n = len(x)
    r = np.correlate(x,x,mode='full')[-n:]
    variance = x.var()
    return r/(variance*np.arange(n,0,-1))

speed_acorr = np.asarray([acorr(speed_detrended[j]) for j in range(speed.shape[0])])

In [ ]:
mean_speed_acorr = speed_acorr.mean(axis=0)

In [ ]:
plt.loglog(mean_speed_acorr)
lags = np.arange(0,1000.)
plt.plot(lags,lags**-.5)
plt.grid()

Yeeee! autocorrelation as $1/\sqrt\tau$

## Acorr of velocity

In [ ]:
def acfNd(v,n_lags=125):
    """
        inspired from vacf.py in this repo
        shape of v: [time,dimensions]
        does dot product in dimensions higher than 1
    """
    
    v = np.asarray(v,dtype=np.float) # make sure we have the right tipe
    lags_n = v.shape[0]
    lags_v = np.unique([int(l) for l in np.logspace(0,np.log10(lags_n),n_lags) if int(l) < lags_n])
    acf = np.zeros((len(lags_v),2))

    variance = np.sum(v*v,axis=1).mean()
    
    for i,lagv in enumerate(lags_v):
        v_corr = np.sum(v[:-lagv]*v[lagv:],axis=1).mean()

        acf[i,0] = lagv
        acf[i,1] = v_corr
        
    return acf/variance

In [ ]:
print (velocity.shape)
velocity =velocity.transpose(0,2,1)
print (velocity.shape)

In [ ]:
def vcorr_particles(velocity):
    try:
        # "async" as long as numpy releases GIL here and there or process pool is used
        from concurrent import futures
        with futures.ProcessPoolExecutor(max_workers=4) as executor:
            vcorr = executor.map(acfNd,velocity)
            vcorr = list(vcorr)
    except ImportError:
        print ("you don't have futures??? ")
        vcorr = [acfNd(j,n_lags=125) for j in velocity]
    return np.asarray(vcorr).mean(axis=0)

In [ ]:
print (velocity.shape)

In [ ]:
vcorr = vcorr_particles(velocity).T

In [ ]:
plt.grid(1)
plt.loglog(vcorr[0],vcorr[1],'-o',ms=2)
plt.ylim(1e-4,1)
plt.xlabel('lags')
plt.ylabel('velocity acf normalized')
plt.plot(vcorr[0],vcorr[0]**-.7*10,label='.7')
plt.plot(vcorr[0],vcorr[0]**-.8*10,label='.8')
plt.plot(vcorr[0],vcorr[0]**-.9*10,label='.9')

plt.legend()

Would be nice to have the vcorr averaged on all particles

In [ ]:
!mkdir G_data

In [ ]:
np.save("G_data/vcor.npy",vcorr.T)